In [1]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# EDA

In [2]:
train = pd.read_csv('./MLProjectData.csv')
test = pd.read_csv('./testData.csv')
train['cat1'] = train.cat1.astype('category')
train['cat2'] = train.cat2.astype('category')

features = train.drop(axis = 1, columns= ['target'])
labels = train['target']
#One hot coding
features_hot = pd.get_dummies(features, columns= ['cat1','cat2'])

In [4]:
train.describe();

### Similarity Matrix
Try to analyze association among those boolean features

In [5]:
from sklearn.metrics import jaccard_similarity_score

In [6]:
cat_cols = [col for col in train.columns if 'cat' in col]
tmp_df = np.full((24, 24), 10)
tmp_df = pd.DataFrame(tmp_df)
tmp_df.columns = cat_cols[2:]
tmp_df.index = cat_cols[2:]

for col1 in cat_cols[2:]:
    for col2 in cat_cols[2:]:
        tmp_df.loc[col1, col2] = jaccard_similarity_score(train[col1], train[col2])

In [7]:
plt.hist(list(tmp_df.values.flat));

### Visualize the features

In [ ]:
#visualization exploration
for i in train:
    if train[i].dtype == 'float64':
        plt.boxplot(train[i])
        plt.title(i)
        plt.show()

In [ ]:
for j in train:
    if train[j].dtype == 'bool':
        (train[j].value_counts()/train.shape[0]).plot(kind = 'bar')
        plt.title(j)
        plt.show()

### Identify the noisy features

In [9]:
from featexp import get_univariate_plots
from featexp import get_trend_stats

In [11]:
for i in train:
    if train[i].dtype == 'float64':
        get_univariate_plots(data=train, target_col='target', features_list=[i], bins=10)

In [ ]:
#select the features with correlation large than 0.6
stats = get_trend_stats(data = train1, target_col= 'target', data_test= test1)
feature_list = stats[stats['Trend_correlation'] > 0.6]['Feature']
feature_list = feature_list[feature_list.str.contains('num')]
feature_list = feature_list.append(pd.Series(['cat1','cat2','cat3','cat4']))
features = train.drop(axis = 1, columns= ['target'])[feature_list]
labels = train['target']
features_hot = pd.get_dummies(features, columns= ['cat1','cat2'])

### Split the training dataset

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size = 0.3, random_state = 123)
train_hot_x, test_hot_x, train_hot_y, test_hot_y = train_test_split(features_hot, labels, test_size = 0.3, random_state = 123)

# Predictive Modeling

In [ ]:
#define a function help use calculate MAE
def mae(x,y):
    """calculate the MAE(mean of absolute error)"""
    mae = abs(x - y).mean()
    return mae

In [ ]:
#define a function help us do 5-fold cross-validation
def get_folds_index(df=None, n_splits=5):
    """Returns dataframe indices corresponding to group"""
    kf = KFold(n_splits=5, random_state= 234)
    fold_ids = []
    ids = df.index
    for train_idx, test_idx in kf.split(X=df):
        fold_ids.append([ids[train_idx], ids[test_idx]])
    return fold_ids

### Our baseline model

In [ ]:
folds_hot = get_folds_index(features_hot)
mae_vc_mean = []
for train_idx_hot, test_idx_hot  in folds_hot:
    """get the train split as well as the test split according to the assignment of 5 validaiton"""
    train_x_cv_hot, train_y_cv_hot = features_hot.iloc[train_idx_hot], labels.iloc[train_idx_hot]
    test_x_cv_hot, test_y_cv_hot = features_hot.iloc[test_idx_hot], labels.iloc[test_idx_hot]
    mae_vc_mean.append(mae(train_y_cv_hot.mean(),test_y_cv_hot))
    
print('The baseline MAE (mean of absolute error)  according to 5-fold validation is ', sum(mae_vc_mean)/5)

### Try linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [ ]:
folds_hot = get_folds_index(features_hot)
mae_vc_linear = []
for train_idx_hot, test_idx_hot  in folds_hot:
    """get the train split as well as the test split according to the assignment of 5 validaiton"""
    train_x_cv_hot, train_y_cv_hot = features_hot.iloc[train_idx_hot], labels.iloc[train_idx_hot]
    test_x_cv_hot, test_y_cv_hot = features_hot.iloc[test_idx_hot], labels.iloc[test_idx_hot]
    reg_vc = LinearRegression().fit(train_x_cv_hot, train_y_cv_hot)
    mae_vc_linear.append(mae(reg_vc.predict(test_x_cv_hot),test_y_cv_hot))
print('The  MAE (mean of absolute error) using linear regression according to 5-fold validation is ', sum(mae_vc_linear)/5)

### Try Lasso

In [ ]:
from sklearn import linear_model
alpha = 100
clf = linear_model.Lasso(alpha=float(alpha) , normalize=True)
lasso = mae(clf.predict(test_hot_x), test_hot_y)
print('The MAE (mean of absolute error) using lasso is ', lasso)

#### Tune the alpha

In [ ]:
from sklearn.linear_model import LassoCV
clf_lasso_cv = LassoCV(cv = 5, random_state= 123, max_iter = 10000).fit(features_hot, labels)
clf_lasso_cv.alpha_

In [ ]:
#the best alph is 30 
alpha = 30
clf = linear_model.Lasso(alpha=float(alpha) , normalize=True)

In [ ]:
mae_lasso_vc = []
for train_idx_hot, test_idx_hot  in folds_hot:
    """get the train split as well as the test split according to the assignment of 5 validaiton"""
    train_x_cv_hot, train_y_cv_hot = features_hot.iloc[train_idx_hot], labels.iloc[train_idx_hot]
    test_x_cv_hot, test_y_cv_hot = features_hot.iloc[test_idx_hot], labels.iloc[test_idx_hot]
    clf.fit(train_x_cv_hot, train_y_cv_hot)
    mae_lasso_vc.append(mae(clf.predict(test_x_cv_hot),test_y_cv_hot))
print('The MAE (mean of absolute error) using lasso according to 5 fold validation is ', sum(mae_lasso_vc)/5)

### Try Lasso with PCA

In [ ]:
pca = PCA(n_components = 3)
pca.fit(np.array(features_hot))
transform_features3 = pd.DataFrame(pca.transform(np.array(features_hot)),columns=['x','y','z'])
trans_train_x, trans_test_x, trans_train_y, trans_test_y = train_test_split(transform_features3, labels, test_size = 0.3, random_state = 123)
clf.fit(trans_train_x, trans_train_y)
mae(clf.predict(trans_test_x),trans_test_y)

### Try Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 120, random_state = 42, criterion='mae',max_features=0.1, n_jobs = -1, min_samples_leaf=20)
rf.fit(train_x, train_y)
mae_random_forest = mae(rf2.predict(test_x),test_y)
print('The random forest MAE (mean of absolute error)  without tuning the model is ', mae_random_forest)

In [12]:
#check the feature importance
rf.feature_importances_

### Try grid search to tune the hyperparameter

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 3)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 15, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [20, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [20, 70]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 30, scoring='neg_mean_absolute_error', 
                              cv = 5, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(features_hot, labels)

In [ ]:
print('The MAE using optimal random forest according to 5 fold validation is ', -rf_random.best_score_)

### Try Xgboost

In [13]:
import xgboost as xgb

In [ ]:
start_xgmodel = xgb.XGBRegressor(objective ='reg:linear',learning_rate= 0.01, n_estimators = 1000, max_depth = 3, subsample = 0.8, colsample_bytree = 1, gamma = 1)
start_xgmodel.fit(train_hot_x, train_hot_y)
XG_mean = mae(start_xgmodel.predict(test_hot_x),test_hot_y)
print('The MAE (mean of absolute error) using a begining xgboost is ', XG_mean)

In [ ]:
#decide the optimized n_estimate
eval_set = [(train_hot_x, train_hot_y), (test_hot_x, test_hot_y)]
eval_metric = ['mae','error']
%time start_xgmodel.fit(train_hot_x, train_hot_y, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

In [ ]:
# the optimal n_estimate is 600
# now we are trying to decide the depth of tree
# we will use randomized search since it is more efficient and time saving

xgb_start = xgb.XGBRegressor(learning_rate= 0.01, n_estimators= 600, objective ='reg:linear', nthread=4, silent=True )
from sklearn.model_selection import StratifiedKFold

max_depth = [int(x) for x in np.linspace(3,6,4)]
min_child_weight = [10,20,40]
gamma = [0.5, 1, 1.5, 2, 5]
subsample = [0.6, 0.8, 1.0]
colsample_bytree = [0.6, 0.8, 1.0]

params_first = {
    'min_child_weight': min_child_weight,
    'max_depth': max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree':colsample_bytree
               }
pprint(params_first)

gsearch = RandomizedSearchCV(xgb_start, param_distributions = params_first, scoring='neg_mean_absolute_error', n_iter = 30, n_jobs=4, cv = 5, verbose=2, random_state=1001)
gsearch.fit(features_hot, labels)

In [ ]:
print('The MAE using optimal xgboost according to 5 fold validation is ', -gsearch.best_score_)